# IEEE-CIS Fraud Detection - data analisys

In [1]:
import sys
import warnings
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, ensemble, metrics, linear_model, svm
#, naive_bayes, metrics, svm, decomposition
#import xgboost 
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

warnings.simplefilter('ignore')
__location__ = sys.path[0] 
output_path = f'{__location__}/Input/ieee-fraud-detection/sample_submission.csv'

Using TensorFlow backend.


In [2]:
#train_identity = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/train_transaction.csv')
#test_identity = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/test_identity.csv')
#test_transaction = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/test_transaction.csv')

In [3]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()

non_num_cols = [train_transaction.columns[n] for n,i in enumerate(train_transaction.dtypes)
                if i not in ('int64','float64')]

for i in non_num_cols:
    #Filling missing string values with most common value
    train_transaction['mod_'+i] = encoder.fit_transform(train_transaction[i].fillna(train_transaction[i].mode()[0]))
    #test_transaction['mod_'+i] = encoder.fit_transform(test_transaction[i].fillna(test_transaction[i].mode()[0])

In [4]:
col_list = [x for x in train_transaction.columns if x not in non_num_cols+['isFraud']]

x_train, y_train, x_valid, y_valid = model_selection.train_test_split(train_transaction[col_list].fillna(0)
                                                                      , train_transaction['isFraud']
                                                                      , test_size=0.9
                                                                     )
scaler = MinMaxScaler(feature_range = (0,1))

scaler.fit(x_train)
x_train = scaler.transform(x_train)
y_train = scaler.transform(y_train)

In [5]:
#sm = SMOTE(random_state = 2)
#x_train, y_train = sm.fit_resample(x_train, y_train)

In [6]:
RF = ensemble.RandomForestClassifier(n_estimators=31
                                     , n_jobs=-1
                                     , max_depth=31
                                     , min_samples_split = 0.3
                                    )
RF.fit(x_train, x_valid)
predictions = RF.predict(y_train)
accuracy = metrics.accuracy_score(predictions, y_valid)
print (f"RF, xx={0}: ", accuracy)


LR = linear_model.LogisticRegression(penalty = 'l2', C = 100)

LR.fit(x_train, x_valid)
predictions = LR.predict(y_train)
accuracy = metrics.accuracy_score(predictions, y_valid)
print (f"LR, c_params_range={100}: ", accuracy)

RF, xx=0:  0.9649304026822908
LR, c_params_range=0.001:  0.9648871277888788
LR, c_params_range=0.01:  0.9652464975559093
LR, c_params_range=0.1:  0.9676510764159357
LR, c_params_range=1:  0.969822347154958
LR, c_params_range=10:  0.9706276364758432
LR, c_params_range=100:  0.9709700725889299


In [ ]:
kernels = ['linear', 'rbf', 'poly']

for kernel in kernels:
    svc = svm.SVC(kernel=kernel).fit(x_train, x_valid)
    predictions = svc.predict(y_train)
    accuracy = metrics.accuracy_score(predictions, y_valid)
    print (f"SVC, kernel={kernel}: ", accuracy)